# **Tugas Praktikum**

Pada tugas pratikum ini Anda akan menggunakan data `"Wisconsin Breast Cancer"`. Data tersebut terdiri dari 569 data yang digunakan untuk mendiagnonis jenis kanker `Malignant (M)` dan `Benign (B)`. Tugas Anda adalah,

1. Pisahkan antara variabel yang dapat digunakan dan variabel yang tidak dapat digunakan.
2. Lakukan proses encoding pada kolom "diagnosis".
3. Lakukan proses standardisasi pada semua kolom yang memiliki nilai numerik.
4. Lakukan proses seleksi fitur. Anda dapat menggunakan SelectKBest.
5. Lakukan proses pengujian dengan model Logistic Regression seperti pada praktikum 1.
6. Anda dapat menggunakan model pipeline untuk mempermudah perkejaan Anda.
7. Berdasarkan hasil analisa Anda, berapa jumlah fitur terbaik yang dapat digunakan? Apa saja fitur tersebut?

In [ ]:
import pandas as pd

from pandas import DataFrame, Series
from sklearn.model_selection import train_test_split # type: ignore
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report # type: ignore
from sklearn.impute import SimpleImputer
from typing import Dict, List

In [ ]:
# 1. Load data
df: DataFrame = pd.read_csv("wbc.csv") # type: ignore

# 2. Pisahkan variabel
df = df.dropna(axis=1, how="all")  # type: ignore # buang kolom kosong sepenuhnya

X: DataFrame = df.drop(columns=["id", "diagnosis"])   # buang kolom id & target
X = X.select_dtypes(include="number")                 # type: ignore # hanya ambil numerik
y: Series = df["diagnosis"].map({"M": 1, "B": 0})

# 3. Bagi data train-test
X_train: DataFrame
X_test: DataFrame
y_train: Series
y_test: Series

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y) # type: ignore

# 4. Pipeline: imputasi -> scaling -> seleksi fitur -> logistic regression
def build_pipeline(k: int) -> Pipeline:
    return Pipeline([
        ("imputer", SimpleImputer(strategy="mean")),   # isi NaN dengan mean
        ("scaler", StandardScaler()),
        ("select", SelectKBest(score_func=f_classif, k=k)),
        ("clf", LogisticRegression(max_iter=1000))
    ])


# 5. Uji dengan berbagai jumlah fitur
results: Dict[int, float] = {}
for k in range(5, X.shape[1] + 1, 5):  # coba per 5 fitur
    pipe: Pipeline = build_pipeline(k)
    pipe.fit(X_train, y_train) # type: ignore
    y_pred: Series = pipe.predict(X_test) # type: ignore
    acc: float = accuracy_score(y_test, y_pred) # type: ignore
    results[k] = acc

# 6. Cari k terbaik
best_k: int = max(results, key=results.get) # type: ignore
best_acc: float = results[best_k]

print("Jumlah fitur terbaik:", best_k) # type: ignore
print("Akurasi:", best_acc)

# 7. Fitur yang terpilih
final_pipe: Pipeline = build_pipeline(best_k) # type: ignore
final_pipe.fit(X_train, y_train) # type: ignore
mask: List[bool] = final_pipe.named_steps["select"].get_support().tolist() # type: ignore
selected_features: List[str] = X.columns[mask].tolist()

print("Fitur terpilih:", selected_features)

# 8. Laporan klasifikasi
y_pred: Series = final_pipe.predict(X_test) # type: ignore
print(classification_report(y_test, y_pred)) # type: ignore

Jumlah fitur terbaik: 20
Akurasi: 0.9824561403508771
Fitur terpilih: ['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean', 'radius_se', 'perimeter_se', 'area_se', 'concave points_se', 'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst', 'concave points_worst', 'symmetry_worst']
              precision    recall  f1-score   support

           0       0.97      1.00      0.99        72
           1       1.00      0.95      0.98        42

    accuracy                           0.98       114
   macro avg       0.99      0.98      0.98       114
weighted avg       0.98      0.98      0.98       114

